In [23]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.join(os.getcwd(), ".."))

from models.sp500.fundamentals import FundamentalStockAnalyzer
import logging
import yfinance as yf
import time

logging.basicConfig(level=logging.CRITICAL)  # or DEBUG for more detail

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Create analyzer using the new async factory method
async def setup_analyzer():
    # Use the async factory method instead of direct constructor
    fas = await FundamentalStockAnalyzer.create()

    # Get tickers (now async)
    tickers = await fas.tickers
    print(f"Loaded {len(tickers)} S&P 500 tickers")
    print(f"First 10 tickers: {tickers[:10]}")

    # Show available field sets (this is still synchronous)
    print(f"\nAvailable field sets: {fas.get_available_field_sets()}")

    return fas


# Run the async setup
fas = await setup_analyzer()

Loaded 503 S&P 500 tickers
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']


In [25]:
fas = await FundamentalStockAnalyzer.create()
tickers = await fas.tickers
print(f"Loaded {len(tickers)} S&P 500 tickers")
print(f"First 10 tickers: {tickers[:10]}")

# Show available field sets (this is still synchronous)
print(f"\nAvailable field sets: {fas.get_available_field_sets()}")

Loaded 503 S&P 500 tickers
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']


In [26]:
# Test single ticker fundamental data (now async)
print("=== Single Ticker Example ===")
result = await fas.get_fundamentals("AAPL", field_set="basic_info")
print(f"AAPL basic info: {result}")

print("\n=== Error Handling Example ===")
error_result = await fas.get_fundamentals("INVALID_TICKER")
print(f"Invalid ticker result: {error_result}")

=== Single Ticker Example ===
AAPL basic info: {'ticker': 'AAPL', 'data': {'shortName': 'Apple Inc.', 'sector': 'Technology', 'industry': 'Consumer Electronics', 'marketCap': 3512722522112, 'currentPrice': 236.7, 'beta': 1.109}}

=== Error Handling Example ===
Invalid ticker result: {'ticker': 'INVALID_TICKER', 'data': {'dividendRate': None, 'oneWeekChangePercent': None, 'sixMonthChangePercent': None, 'targetMeanPrice': None, 'returnOnAssets': None, 'debtToEquity': None, 'sector': None, 'marketCap': None, 'trailingEps': None, 'forwardPE': None, 'totalCash': None, 'totalCashPerShare': None, 'industry': None, 'currentPrice': None, 'earningsGrowth': None, 'beta': None, 'operatingCashflow': None, 'threeMonthChangePercent': None, 'priceToBook': None, 'shortName': None, 'trailingAnnualDividendRate': None, 'enterpriseValue': None, 'ebitda': None, 'returnOnEquity': None, 'freeCashflow': None, 'recommendationKey': None, 'forwardEps': None, 'oneMonthChangePercent': None, 'annualChangePercent': N

In [27]:
# Test multiple tickers (small sample) - now with async and concurrency control
print("=== Multiple Tickers Example ===")
sample_tickers = ["AAPL", "MSFT", "GOOGL"]

start_time = time.time()

results = await fas.get_fundamentals_concurrent(
    tickers=sample_tickers,
    field_set="basic_info",  # Using basic_info since value_analysis might not exist
    max_concurrent=3,  # Process all 3 concurrently
)

end_time = time.time()
print(f"Fetched {len(sample_tickers)} tickers in {end_time - start_time:.2f} seconds")

for result in results:
    if "error" not in result:
        ticker = result["ticker"]
        data = result["data"]
        short_name = data.get("shortName", "Unknown")
        market_cap = data.get("marketCap", "N/A")
        print(f"{ticker}: {short_name} - Market Cap: {market_cap}")
    else:
        print(f"{result['ticker']}: Error - {result['error']}")

print("\n=== Configuration Examples ===")
print(f"Cache days setting: {fas.config.cache_days}")
print(f"Data source URL: {fas.config.data_url}")

# Show different field sets
basic_fields = fas.get_fields("basic_info")
all_field_sets = fas.get_available_field_sets()
print(f"Basic info fields ({len(basic_fields)}): {basic_fields}")
print(f"All available field sets: {all_field_sets}")
if "value_analysis" in all_field_sets:
    value_fields = fas.get_fields("value_analysis")
    print(f"Value analysis fields ({len(value_fields)}): {value_fields}")

=== Multiple Tickers Example ===
Fetched 3 tickers in 1.04 seconds
AAPL: Apple Inc. - Market Cap: 3512722522112
MSFT: Microsoft Corporation - Market Cap: 3830758244352
GOOGL: Alphabet Inc. - Market Cap: 2915379970048

=== Configuration Examples ===
Cache days setting: 30
Data source URL: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Basic info fields (6): ['shortName', 'sector', 'industry', 'marketCap', 'currentPrice', 'beta']
All available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']
Value analysis fields (6): ['priceToBook', 'trailingPE', 'forwardPE', 'pegRatio', 'priceToSalesTrailing12Months', 'enterpriseToEbitda']


In [28]:
stock = yf.Ticker("V")
hist = stock.history(period="1y")
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-09-16 00:00:00-04:00,286.041238,288.871369,285.733402,288.454315,4458900,0.0,0.0
2024-09-17 00:00:00-04:00,288.871393,291.026274,288.057104,289.526794,5135700,0.0,0.0
2024-09-18 00:00:00-04:00,289.804785,289.983522,286.120666,286.468231,5890000,0.0,0.0
2024-09-19 00:00:00-04:00,289.060034,289.447329,280.897356,283.250824,10382100,0.0,0.0
2024-09-20 00:00:00-04:00,283.509039,284.998578,281.274729,282.784119,27810900,0.0,0.0


In [29]:
# Performance comparison: Sequential vs Concurrent processing
print("=== Performance Comparison ===")

# Test with more tickers to see the performance difference
test_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "NVDA"]

print(f"Testing with {len(test_tickers)} tickers: {', '.join(test_tickers)}")

# Sequential processing (max_concurrent=1)
print("\n1. Sequential processing:")
start = time.time()
sequential_results = await fas.get_fundamentals_concurrent(
    tickers=test_tickers, field_set="basic_info", max_concurrent=1
)
sequential_time = time.time() - start
print(f"   Time: {sequential_time:.2f} seconds")

# Concurrent processing (max_concurrent=5)
print("\n2. Concurrent processing:")
start = time.time()
concurrent_results = await fas.get_fundamentals_concurrent(
    tickers=test_tickers, field_set="basic_info", max_concurrent=5
)
concurrent_time = time.time() - start
print(f"   Time: {concurrent_time:.2f} seconds")

# Calculate speedup
if concurrent_time > 0:
    speedup = sequential_time / concurrent_time
    print(f"\n🚀 Speedup: {speedup:.1f}x faster with concurrency!")
    print(f"   Time saved: {sequential_time - concurrent_time:.1f} seconds")

# Show some results
print(f"\nSuccessfully fetched data for {len(concurrent_results)} tickers:")
for result in concurrent_results[:3]:  # Show first 3
    if "error" not in result:
        ticker = result["ticker"]
        market_cap = result["data"].get("marketCap", "N/A")
        print(f"  {ticker}: Market Cap = {market_cap}")
print("  ...")

=== Performance Comparison ===
Testing with 8 tickers: AAPL, MSFT, GOOGL, AMZN, TSLA, META, NFLX, NVDA

1. Sequential processing:
   Time: 2.71 seconds

2. Concurrent processing:
   Time: 0.43 seconds

🚀 Speedup: 6.3x faster with concurrency!
   Time saved: 2.3 seconds

Successfully fetched data for 8 tickers:
  AAPL: Market Cap = 3512722522112
  MSFT: Market Cap = 3830758244352
  GOOGL: Market Cap = 2915379970048
  ...


In [63]:
# Fetch valuation metrics for the test tickers sequentially (no concurrency)
# This uses the "valuation_metrics" field set, which is available in all_field_sets

sequential_results = await fas.get_fundamentals_concurrent(
    tickers=test_tickers,
    field_set="cash_flow_metrics",
)

# Display results in a user-friendly format
for result in sequential_results:
    ticker = result["ticker"]
    data = result.get("data", {})
    if "error" in result:
        print(f"{ticker}: Error - {result['error']}")
    else:
        print(f"\n{ticker} Valuation Metrics:")
        for field, value in data.items():
            print(f"  {field}: {value}")


AAPL Valuation Metrics:
  operatingCashflow: 108564996096
  freeCashflow: 94873747456
  totalCash: 55372001280
  totalCashPerShare: 3.731

MSFT Valuation Metrics:
  operatingCashflow: 136162000896
  freeCashflow: 61070376960
  totalCash: 94564999168
  totalCashPerShare: 12.722

GOOGL Valuation Metrics:
  operatingCashflow: 133707997184
  freeCashflow: 49786499072
  totalCash: 95147999232
  totalCashPerShare: 7.867

AMZN Valuation Metrics:
  operatingCashflow: 121136996352
  freeCashflow: 31024001024
  totalCash: 93180002304
  totalCashPerShare: 8.737

TSLA Valuation Metrics:
  operatingCashflow: 15765000192
  freeCashflow: 1339624960
  totalCash: 36781998080
  totalCashPerShare: 11.404

META Valuation Metrics:
  operatingCashflow: 102299000832
  freeCashflow: 31991250944
  totalCash: 47070998528
  totalCashPerShare: 18.737

NFLX Valuation Metrics:
  operatingCashflow: 9070451712
  freeCashflow: 22694656000
  totalCash: 8390519808
  totalCashPerShare: 19.746

NVDA Valuation Metrics:
  

In [31]:
# Recreate analyzer to pick up config changes
print("=== Recreating analyzer with updated config ===")
fas = await FundamentalStockAnalyzer.create()

# Check available field sets
print(f"Available field sets: {fas.get_available_field_sets()}")

# Check if our new percentage_changes field group is available
try:
    percentage_fields = fas.get_fields("percentage_changes")
    print(f"Percentage change fields: {percentage_fields}")
except ValueError as e:
    print(f"Error: {e}")

# Check if price_performance field set is available
try:
    price_performance_fields = fas.get_fields("price_performance")
    print(f"Price performance fields: {price_performance_fields}")
except ValueError as e:
    print(f"Error: {e}")

=== Recreating analyzer with updated config ===
Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']
Percentage change fields: ['annualChangePercent', 'sixMonthChangePercent', 'threeMonthChangePercent', 'oneMonthChangePercent', 'oneWeekChangePercent']
Price performance fields: ['currentPrice', 'annualChangePercent', 'sixMonthChangePercent', 'threeMonthChangePercent', 'oneMonthChangePercent', 'oneWeekChangePercent']


In [32]:
# Test the percentage change calculations
print("=== Testing Percentage Change Calculations ===")

# Test with AAPL
print("Testing AAPL percentage changes:")
result = await fas.get_fundamentals("AAPL", field_set="percentage_changes")
if "error" not in result:
    for field, value in result["data"].items():
        print(f"  {field}: {value}%")
else:
    print(f"Error: {result['error']}")

print("\n" + "=" * 50)

# Test with price_performance field set for MSFT
print("Testing MSFT price performance:")
result = await fas.get_fundamentals("MSFT", field_set="price_performance")
if "error" not in result:
    for field, value in result["data"].items():
        if field == "currentPrice":
            print(f"  {field}: ${value}")
        else:
            print(f"  {field}: {value}%")
else:
    print(f"Error: {result['error']}")

print("\n" + "=" * 50)

# Test multiple tickers with percentage changes
print("Testing multiple tickers with percentage changes:")
sample_tickers = ["AAPL", "GOOGL"]
results = await fas.get_fundamentals_concurrent(
    tickers=sample_tickers, field_set="percentage_changes", max_concurrent=2
)

for result in results:
    ticker = result["ticker"]
    if "error" not in result:
        print(f"\n{ticker}:")
        annual = result["data"].get("annualChangePercent")
        one_week = result["data"].get("oneWeekChangePercent")
        print(f"  Annual Change: {annual}%")
        print(f"  One Week Change: {one_week}%")
    else:
        print(f"{ticker}: Error - {result['error']}")

=== Testing Percentage Change Calculations ===
Testing AAPL percentage changes:
  annualChangePercent: 4.21%
  sixMonthChangePercent: 8.71%
  threeMonthChangePercent: 17.89%
  oneMonthChangePercent: 3.93%
  oneWeekChangePercent: 1.12%

Testing MSFT price performance:
  currentPrice: $515.36
  annualChangePercent: 19.29%
  sixMonthChangePercent: 32.18%
  threeMonthChangePercent: 8.13%
  oneMonthChangePercent: 1.77%
  oneWeekChangePercent: 1.07%

Testing multiple tickers with percentage changes:

AAPL:
  Annual Change: 4.21%
  One Week Change: 1.12%

GOOGL:
  Annual Change: 54.48%
  One Week Change: 4.58%


In [33]:
# Final integration test - combining traditional fundamentals with percentage changes
print("=== Integration Test: Traditional + Percentage Changes ===")

# Test mixing basic info with percentage changes
result = await fas.get_fundamentals("TSLA", field_set="basic_info")
print("TSLA basic info (without percentage changes):")
for field, value in result["data"].items():
    print(f"  {field}: {value}")

print("\n" + "-" * 40)

# Now test basic_info + specific percentage fields by using "all" field set
result = await fas.get_fundamentals("TSLA", field_set="all")
print("TSLA sample of all fields (showing some basics + percentage changes):")
data = result["data"]
sample_fields = [
    "shortName",
    "currentPrice",
    "marketCap",
    "annualChangePercent",
    "sixMonthChangePercent",
    "oneWeekChangePercent",
]

for field in sample_fields:
    value = data.get(field)
    if field in [
        "annualChangePercent",
        "sixMonthChangePercent",
        "oneWeekChangePercent",
    ]:
        print(f"  {field}: {value}%")
    elif field == "currentPrice":
        print(f"  {field}: ${value}")
    else:
        print(f"  {field}: {value}")

print(
    f"\n✅ Successfully integrated {len([f for f in data.keys() if 'ChangePercent' in f])} percentage change fields!"
)
print("✅ All percentage change calculations completed successfully!")
print("✅ Configuration system updated with new field groups!")
print("✅ Async implementation maintains performance!")

=== Integration Test: Traditional + Percentage Changes ===
TSLA basic info (without percentage changes):
  shortName: Tesla, Inc.
  sector: Consumer Cyclical
  industry: Auto Manufacturers
  marketCap: 1322563534848
  currentPrice: 410.04
  beta: 2.065

----------------------------------------
TSLA sample of all fields (showing some basics + percentage changes):
  shortName: Tesla, Inc.
  currentPrice: $410.04
  marketCap: 1322563534848
  annualChangePercent: 72.1%
  sixMonthChangePercent: 64.87%
  oneWeekChangePercent: 3.56%

✅ Successfully integrated 5 percentage change fields!
✅ All percentage change calculations completed successfully!
✅ Configuration system updated with new field groups!
✅ Async implementation maintains performance!


## Stock Filtering with Custom Logic in S&P 500 Fundamentals Model

In [38]:
fas = await FundamentalStockAnalyzer.create()
print("=== Fetching full S&P 500 fundamentals concurrently ===")
sp500_data = await fas.get_fundamentals_concurrent(field_set="all", max_concurrent=10)
print(f"Fetched data for {len(sp500_data)} S&P 500 companies.")

=== Fetching full S&P 500 fundamentals concurrently ===
Fetched data for 503 S&P 500 companies.


### Strict Criteria for Stock Selection
**Selection Criteria:**

- Annual Change ≤ **-25%**
- Six Month Change ≤ **0%**
- Three Month Change ≤ **0%**
- One Month Change ≤ **0%**
- One Week Change ≤ **0%**
- Cash Flow > **$300,000,000**
- Free Cash Flow > **$300,000,000**
- EBITDA > **0**
- Profit Margins > **0**
- Revenue Growth > **-10%**
- EBITDA Margins > **0**
- Operating Margins > **0**
- Return on Assets > **0**

These strict filters are designed to identify S&P 500 stocks with significant recent declines across multiple timeframes, while ensuring the company maintains strong positive cash flow. This approach helps surface potential turnaround candidates with robust financial health.


In [76]:
# Get list of stocks that fit the strict criteria and display them
print("=== Filtering stocks that fit the strict criteria ===")
filtered_stocks = [
    stock
    for stock in sp500_data
    if stock["data"].get("annualChangePercent") is not None
    and stock["data"]["annualChangePercent"] <= -25
    and stock["data"].get("sixMonthChangePercent") is not None
    and stock["data"]["sixMonthChangePercent"] <= 0
    and stock["data"].get("threeMonthChangePercent") is not None
    and stock["data"]["threeMonthChangePercent"] <= 0
    and stock["data"].get("oneMonthChangePercent") is not None
    and stock["data"]["oneMonthChangePercent"] <= 0
    and stock["data"].get("oneWeekChangePercent") is not None
    and stock["data"]["oneWeekChangePercent"] <= 0
    and stock["data"].get("operatingCashflow") is not None
    and stock["data"]["operatingCashflow"] > 300000000
    and stock["data"].get("freeCashflow") is not None
    and stock["data"]["freeCashflow"] > 300000000
    and stock["data"].get("ebitda") is not None
    and stock["data"]["ebitda"] > 0
    and stock["data"].get("profitMargins") is not None
    and stock["data"]["profitMargins"] > 0
    and stock["data"].get("revenueGrowth") is not None
    and stock["data"]["revenueGrowth"] > -10
    and stock["data"].get("ebitdaMargins") is not None
    and stock["data"]["ebitdaMargins"] > 0
    and stock["data"].get("operatingMargins") is not None
    and stock["data"]["operatingMargins"] > 0
    and stock["data"].get("returnOnAssets") is not None
    and stock["data"]["returnOnAssets"] > 0
]
print(f"Found {len(filtered_stocks)} stocks that fit the strict criteria:")
for stock in filtered_stocks:
    ticker = stock["ticker"]
    name = stock["data"].get("shortName", "N/A")
    annual_change = stock["data"].get("annualChangePercent", "N/A")
    six_month_change = stock["data"].get("sixMonthChangePercent", "N/A")
    three_month_change = stock["data"].get("threeMonthChangePercent", "N/A")
    one_month_change = stock["data"].get("oneMonthChangePercent", "N/A")
    one_week_change = stock["data"].get("oneWeekChangePercent", "N/A")
    cash_flow = stock["data"].get("cashFlow", "N/A")
    free_cash_flow = stock["data"].get("freeCashFlow", "N/A")
    ebitda = stock["data"].get("ebitda", "N/A")
    profit_margins = stock["data"].get("profitMargins", "N/A")
    revenue_growth = stock["data"].get("revenueGrowth", "N/A")
    ebitda_margins = stock["data"].get("ebitdaMargins", "N/A")
    operating_margins = stock["data"].get("operatingMargins", "N/A")
    return_on_assets = stock["data"].get("returnOnAssets", "N/A")

    print(f"\nTicker: {ticker} - {name}")
    print(f"  Annual Change: {annual_change}%")
    print(f"  6-Month Change: {six_month_change}%")
    print(f"  3-Month Change: {three_month_change}%")
    print(f"  1-Month Change: {one_month_change}%")
    print(f"  1-Week Change: {one_week_change}%")
    print(f"  Cash Flow: ${cash_flow}")
    print(f"  Free Cash Flow: ${free_cash_flow}")
    print(f"  EBITDA: ${ebitda}")
    print(f"  Profit Margins: {profit_margins}")
    print(f"  Revenue Growth: {revenue_growth}%")
    print(f"  EBITDA Margins: {ebitda_margins}")
    print(f"  Operating Margins: {operating_margins}")
    print(f"  Return on Assets: {return_on_assets}%")

=== Filtering stocks that fit the strict criteria ===
Found 19 stocks that fit the strict criteria:

Ticker: ACN - Accenture plc
  Annual Change: -27.95%
  6-Month Change: -21.31%
  3-Month Change: -16.23%
  1-Month Change: -8.24%
  1-Week Change: -0.28%
  Cash Flow: $N/A
  Free Cash Flow: $N/A
  EBITDA: $11767775232
  Profit Margins: 0.11607
  Revenue Growth: 0.077%
  EBITDA Margins: 0.17184
  Operating Margins: 0.16825001
  Return on Assets: 0.112449996%

Ticker: ADBE - Adobe Inc.
  Annual Change: -33.54%
  6-Month Change: -10.37%
  3-Month Change: -7.91%
  1-Month Change: -4.14%
  1-Week Change: -0.65%
  Cash Flow: $N/A
  Free Cash Flow: $N/A
  EBITDA: $9059999744
  Profit Margins: 0.30012
  Revenue Growth: 0.107%
  EBITDA Margins: 0.39084
  Operating Margins: 0.36289
  Return on Assets: 0.18094%

Ticker: ALGN - Align Technology, Inc.
  Annual Change: -48.17%
  6-Month Change: -22.27%
  3-Month Change: -27.91%
  1-Month Change: -12.38%
  1-Week Change: -0.38%
  Cash Flow: $N/A
  Fre

### Relaxed Criteria for Stock Selection
**Selection Criteria:**
Exactly one of the following fields must be between 0 and 3% and the others must be less than or equal to 0%:
- Six Month Change
- Three Month Change
- One Month Change
- One Week Change

All other criteria remain the same:
- Annual Change ≤ **-25%**
- Cash Flow > **$300,000,000**
- Free Cash Flow > **$300,000,000**
- EBITDA > **0**
- Profit Margins > **0**
- Revenue Growth > **-10%**
- EBITDA Margins > **0**
- Operating Margins > **0**
- Return on Assets > **0**

This relaxed filter aims to identify stocks that have experienced slight positive movements in one of the shorter-term timeframes, while still showing overall weakness in the others. This can help find stocks that may be starting to stabilize or recover after a period of decline.



In [75]:
# Modified version that captures BOTH scenarios:
# 1. Exactly one short-term field in (0, 3%] and others <= 0 (original logic)
# 2. ALL short-term fields <= 0 (new addition)

relaxed_filtered_stocks = []
all_negative_stocks = []  # New: track stocks with all periods <= 0

for i, stock in enumerate(sp500_data, start=1):
    data = stock["data"]
    # Check annual change and financial criteria (same as before)
    if (
        data.get("annualChangePercent") is not None
        and data["annualChangePercent"] <= -25
        and data.get("operatingCashflow") is not None
        and data["operatingCashflow"] > 300000000
        and data.get("freeCashflow") is not None
        and data["freeCashflow"] > 300000000
        and data.get("ebitda") is not None
        and data["ebitda"] > 0
        and data.get("profitMargins") is not None
        and data["profitMargins"] > 0
        and data.get("revenueGrowth") is not None
        and data["revenueGrowth"] > -10
        and data.get("ebitdaMargins") is not None
        and data["ebitdaMargins"] > 0
        and data.get("operatingMargins") is not None
        and data["operatingMargins"] > 0
        and data.get("returnOnAssets") is not None
        and data["returnOnAssets"] > 0
    ):
        # Check short-term change criteria
        change_fields = [
            "sixMonthChangePercent",
            "threeMonthChangePercent",
            "oneMonthChangePercent",
            "oneWeekChangePercent",
        ]
        values = []
        valid = True
        for field in change_fields:
            val = data.get(field)
            if val is None:
                valid = False
                break
            values.append(val)

        if valid:
            # Original logic: exactly one in (0, 3] and others <= 0
            in_range = [0 < v <= 3 for v in values]
            others_nonpos = [v <= 0 for i, v in enumerate(values) if not in_range[i]]

            # NEW: Check if ALL short-term periods are <= 0
            all_nonpositive = all(v <= 0 for v in values)

            if sum(in_range) == 1 and all(others_nonpos):
                relaxed_filtered_stocks.append(stock)
            elif all_nonpositive:  # NEW condition
                all_negative_stocks.append(stock)

# Display results
print(
    f"🎯 Stocks with exactly one positive short-term period (0-3%): {len(relaxed_filtered_stocks)}"
)
print(f"📉 Stocks with ALL short-term periods <= 0%: {len(all_negative_stocks)}")
print(
    f"🔢 Total qualifying stocks: {len(relaxed_filtered_stocks) + len(all_negative_stocks)}"
)

# Show stocks with one positive period
if relaxed_filtered_stocks:
    print("\n=== Stocks with one positive short-term period (0-3%) ===")
    for stock in relaxed_filtered_stocks:
        ticker = stock["ticker"]
        name = stock["data"].get("shortName", "N/A")
        annual = stock["data"].get("annualChangePercent", "N/A")
        six_month = stock["data"].get("sixMonthChangePercent", "N/A")
        three_month = stock["data"].get("threeMonthChangePercent", "N/A")
        one_month = stock["data"].get("oneMonthChangePercent", "N/A")
        one_week = stock["data"].get("oneWeekChangePercent", "N/A")

        print(f"   {ticker} - {name}")
        print(
            f"     Annual: {annual}%, 6M: {six_month}%, 3M: {three_month}%, 1M: {one_month}%, 1W: {one_week}%"
        )

# Show all negative stocks
if all_negative_stocks:
    print("\n=== Stocks with ALL short-term periods <= 0% ===")
    for stock in all_negative_stocks:
        ticker = stock["ticker"]
        name = stock["data"].get("shortName", "N/A")
        annual = stock["data"].get("annualChangePercent", "N/A")
        six_month = stock["data"].get("sixMonthChangePercent", "N/A")
        three_month = stock["data"].get("threeMonthChangePercent", "N/A")
        one_month = stock["data"].get("oneMonthChangePercent", "N/A")
        one_week = stock["data"].get("oneWeekChangePercent", "N/A")

        print(f"   {ticker} - {name}")
        print(
            f"     Annual: {annual}%, 6M: {six_month}%, 3M: {three_month}%, 1M: {one_month}%, 1W: {one_week}%"
        )

if len(relaxed_filtered_stocks) == 0 and len(all_negative_stocks) == 0:
    print("😢 No stocks found that meet either criteria.")

🎯 Stocks with exactly one positive short-term period (0-3%): 4
📉 Stocks with ALL short-term periods <= 0%: 19
🔢 Total qualifying stocks: 23

=== Stocks with one positive short-term period (0-3%) ===
   LULU - lululemon athletica inc.
     Annual: -39.08%, 6M: -50.41%, 3M: -29.67%, 1M: -22.93%, 1W: 0.07%
   MOH - Molina Healthcare Inc
     Annual: -49.89%, 6M: -43.18%, 3M: -40.1%, 1M: 0.96%, 1W: -2.64%
   RVTY - Revvity, Inc.
     Annual: -31.46%, 6M: -21.59%, 3M: -11.5%, 1M: -10.78%, 1W: 1.36%
   TTD - The Trade Desk, Inc.
     Annual: -58.41%, 6M: -19.13%, 3M: -33.26%, 1M: -14.4%, 1W: 0.86%

=== Stocks with ALL short-term periods <= 0% ===
   ACN - Accenture plc
     Annual: -27.95%, 6M: -21.31%, 3M: -16.23%, 1M: -8.24%, 1W: -0.28%
   ADBE - Adobe Inc.
     Annual: -33.54%, 6M: -10.37%, 3M: -7.91%, 1M: -4.14%, 1W: -0.65%
   ALGN - Align Technology, Inc.
     Annual: -48.17%, 6M: -22.27%, 3M: -27.91%, 1M: -12.38%, 1W: -0.38%
   CDW - CDW Corporation
     Annual: -26.57%, 6M: -0.77%, 3M

### Past results with relaxed criteria
===Filtering for stocks that meet the relaxed criteria===
🎉 Found 4 stocks out of 503 analyzed that fit the relaxed criteria:
   - LULU - lululemon athletica inc.
   - MOH - Molina Healthcare Inc
   - RVTY - Revvity, Inc.
   - TTD - The Trade Desk, Inc.